# Building a song recommender


In [38]:
import graphlab as gl

In [2]:
# Limit number of worker processes. This preserves system memory, which prevents hosted notebooks from crashing.
graphlab.set_runtime_config('GRAPHLAB_DEFAULT_NUM_PYLAMBDA_WORKERS', 4)

[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1547710849.log


This non-commercial license of GraphLab Create for academic use is assigned to ehgus1206@gmail.com and will expire on January 16, 2020.


# Quiz 

1. 아래 가수 중 unique한 사용자 수가 가장 많은 가수는?
('Kanye West', 'Foo Fighters', 'Taylor Swift' , 'Lady GaGa')
2. 가장 많은 조회수를 가진 노래는?
힌트: https://turi.com/products/create/docs/generated/graphlab.SFrame.groupby.html

3. 위 2번 노래를 들은 사용자 중 한 명을 선택하고 노래를 추천해 보자.
3. 위 2번 노래와 유사한 노래는?
4. 가장 인기가 없는 가수는?

## 1
답 : Taylor Swift

In [88]:
dat = song_data.groupby(['artist', 'user_id'], \
                        gl.aggregate.COUNT('user_id')).groupby('artist', {'distinct': gl.aggregate.COUNT('user_id'), \
                                                                          'count': gl.aggregate.SUM('Count')})

In [87]:
singers = ['Kanye West', 'Foo Fighters', 'Taylor Swift' , 'Lady GaGa']
for singer in singers : 
    print(dat[dat['artist'] == singer])

+------------+----------+-------+
|   artist   | distinct | count |
+------------+----------+-------+
| Kanye West |   2522   |  3775 |
+------------+----------+-------+
[? rows x 3 columns]
Note: Only the head of the SFrame is printed. This SFrame is lazily evaluated.
You can use sf.materialize() to force materialization.
+--------------+----------+-------+
|    artist    | distinct | count |
+--------------+----------+-------+
| Foo Fighters |   2055   |  3429 |
+--------------+----------+-------+
[? rows x 3 columns]
Note: Only the head of the SFrame is printed. This SFrame is lazily evaluated.
You can use sf.materialize() to force materialization.
+--------------+----------+-------+
|    artist    | distinct | count |
+--------------+----------+-------+
| Taylor Swift |   3246   |  6227 |
+--------------+----------+-------+
[? rows x 3 columns]
Note: Only the head of the SFrame is printed. This SFrame is lazily evaluated.
You can use sf.materialize() to force materialization.
+----

In [98]:
import graphlab.aggregate as agg
user_count_unique = song_data.groupby(key_columns='artist', operations={'unique_user_count': agg.COUNT_DISTINCT('user_id')})
user_count_unique.sort('unique_user_count', ascending=False)
artists = ['Kanye West', 'Foo Fighters', 'Taylor Swift' , 'Lady GaGa']
user_count_unique.filter_by(artists, 'artist')

artist,unique_user_count
Kanye West,2522
Lady GaGa,2928
Taylor Swift,3246
Foo Fighters,2055


## 2
답 : You're The One

In [99]:
song_count = song_data.groupby(key_columns=['title','song_id'], operations={'song_count': agg.SUM('listen_count')})
song_count.sort('song_count', ascending=False)

song_id,title,song_count
SOBONKR12A58A7A7E0,You're The One,40619
SOAUWYT12A81C206F1,Undo,36059
SOSXLTC12AF72A7F54,Revelry,30391
SOEGIYH12A6D4FC0E3,Horn Concerto No. 4 in Eflat K495: II. Romance ...,21953
SOFRQTD12A81C233C0,Sehr kosmisch,21646
SOAXGDH12A8C13F8A1,Dog Days Are Over (RadioEdit) ...,15889
SONYKOW12AB01849C9,Secrets,14149
SOVDSJC12A58A7A271,Ain't Misbehavin,14029
SOUFTBI12AB0183F65,Invalid,13701
SOHTKMO12AB01843B0,Catch You Baby (StevePitron & Max Sanna Radio ...,12506


## 3. 
2번의 노래를 들은 사람 'b80344d063b5ccb3212f76538f3d9e43d87dca9e', 아래는 추천 결과 

1st rank song : Meadowlarks - Fleet Foxes

In [100]:
song_data['title' == 'You\'re The One']

{'artist': 'Jack Johnson',
 'listen_count': 1,
 'song': 'The Cove - Jack Johnson',
 'song_id': 'SOAKIMP12A8C130995',
 'title': 'The Cove',
 'user_id': 'b80344d063b5ccb3212f76538f3d9e43d87dca9e'}

In [101]:
personalized_model.recommend(users=['b80344d063b5ccb3212f76538f3d9e43d87dca9e'])

user_id,song,score,rank
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,Meadowlarks - Fleet Foxes,0.0248072429707,1
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,Quiet Houses - FleetFoxes ...,0.0240329645182,2
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,Heard Them Stirring -Fleet Foxes ...,0.0203885561542,3
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,Tiger Mountain PeasantSong - Fleet Foxes ...,0.0199806752958,4
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,Your Protector - FleetFoxes ...,0.0193978893129,5
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,Oliver James - FleetFoxes ...,0.0190611293441,6
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,Great Indoors - JohnMayer ...,0.0149489750988,7
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,Innocent Son - FleetFoxes ...,0.0148925859677,8
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,White Winter Hymnal -Fleet Foxes ...,0.0148194040123,9
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,City Love - John Mayer,0.0138473055865,10


## 4.
2번의 노래와 유사한 노래 -> Secrets - OneRepublic

In [102]:
personalized_model.get_similar_items(['Sehr kosmisch - Harmonia'])

song,similar,score,rank
Sehr kosmisch - Harmonia,Secrets - OneRepublic,0.180812895298,1
Sehr kosmisch - Harmonia,Dog Days Are Over (RadioEdit) - Florence + The ...,0.17366361618,2
Sehr kosmisch - Harmonia,Hey_ Soul Sister - Train,0.146540880203,3
Sehr kosmisch - Harmonia,Fireflies - CharttraxxKaraoke ...,0.143619537354,4
Sehr kosmisch - Harmonia,OMG - Usher featuringwill.i.am ...,0.11647772789,5
Sehr kosmisch - Harmonia,Marry Me - Train,0.112236261368,6
Sehr kosmisch - Harmonia,Undo - Björk,0.103723704815,7
Sehr kosmisch - Harmonia,Revelry - Kings Of Leon,0.0907534360886,8
Sehr kosmisch - Harmonia,You're The One - DwightYoakam ...,0.0897599458694,9
Sehr kosmisch - Harmonia,Drop The World - LilWayne / Eminem ...,0.0892943143845,10


## 5.
인기 없는 노래 -> Hubcap - Sleater-kinney & Younger Than Springtime - William Tabbert- 

In [103]:
song_data.groupby('song', gl.aggregate.COUNT('listen_count')).sort('Count') 

song,Count
Hubcap - Sleater-kinney,12
Younger Than Springtime -William Tabbert ...,12
Trahison - Vitalic,15
Marching Theme - NeutralMilk Hotel ...,15
Bendable Poseable - HotChip ...,16
Accidntel Deth (AlbumVersion) - Rilo Kiley ...,16
Made In The Dark - HotChip ...,16
Jumpers (Album) -Sleater-kinney ...,16
Tchaparian - Hot Chip,17
Taste Test - Sleater-kinney ...,17
